In [2]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
def datapre(df):
    df1=df.drop(['Id','PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1)#刪除部分欄位
    df2=pd.get_dummies(df1)#on hot
    df2=df2.fillna(df2.mean())#補值
    df3=df2.reset_index()
    df3=df3.drop(['index'],axis=1)
    return df3
train_df=pd.read_csv(r'train.csv')
test_df=pd.read_csv(r'test.csv')
df1=datapre(train_df)
y=df1['SalePrice'].values
X=df1.drop(['SalePrice'], axis=1).values
ss = StandardScaler()
std_X = ss.fit_transform(X)

In [6]:
df1['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [8]:
xgb.XGBRegressor()

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=None,
       min_child_weight=None, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=None, num_parallel_tree=None,
       objective='reg:squarederror', random_state=None, reg_alpha=None,
       reg_lambda=None, scale_pos_weight=None, subsample=None,
       tree_method=None, validate_parameters=None, verbosity=None)

In [16]:
model = xgb.XGBRegressor()
parameters = {
    'max_depth': [ 0,2 ],
    'learning_rate':[0.1,0.05],
    'n_estimators':[200,190],
    'gamma':[0.2,0.3,0.4]
}
RandomizedSearchCV_model = RandomizedSearchCV(model,
                          parameters,
                          cv=3,#kf
                          scoring='r2',
                          return_train_score='cv_results_',#存放所有實驗
                          verbose=1,
                          n_jobs=-1)
RandomizedSearchCV_model.fit(std_X, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.4s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=None,
       min_child_we..._pos_weight=None, subsample=None,
       tree_method=None, validate_parameters=None, verbosity=None),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'max_depth': [0, 2], 'learning_rate': [0.1, 0.05], 'n_estimators': [200, 190], 'gamma': [0.2, 0.3, 0.4]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='cv_results_', scoring='r2', verbose=1)

In [15]:
print(RandomizedSearchCV_model.best_score_)
print(RandomizedSearchCV_model.best_params_)

0.8883523579830855
{'n_estimators': 200, 'max_depth': 2, 'learning_rate': 0.1, 'gamma': 0.3}


In [17]:
clf = xgb.XGBRegressor(n_estimators=200,max_depth=2,learning_rate=0.1,gamma=0.3)
clf.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.3, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=200, n_jobs=6, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

In [18]:
dfcon=pd.concat([train_df.drop(['SalePrice'], axis=1),test_df])
dfcon1=datapre(dfcon)
test_df1=dfcon1.iloc[1460:2919]
X_test=test_df1.values
y_pred = clf.predict(X_test)
y_pred

array([118988.44, 163017.3 , 179579.89, ..., 168525.4 , 121439.5 ,
       243648.56], dtype=float32)

In [19]:
y_pred_df=pd.DataFrame(y_pred)
y_pred_df1=pd.DataFrame(test_df['Id'])
y_pred_df1['SalePrice']=y_pred_df[0]
y_pred_df1.to_csv(r'leonpre_byXGb.csv',index=False)